<a href="https://colab.research.google.com/github/Sairamsparks2003/Vocalysis/blob/main/Vocalysis_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install necessary packages one by one
!pip install pytube
!pip install pydub
!pip install librosa
!pip install scikit-learn
!pip install tensorflow
!pip install yt-dlp



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.6 MB/s eta 0:00:00


In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import librosa
import librosa.display
from pytube import YouTube
from pydub import AudioSegment
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [6]:
import yt_dlp

In [7]:
def download_youtube_audio(url, output_path):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        filename = ydl.prepare_filename(info)
        return os.path.splitext(filename)[0] + '.mp3'

In [8]:
def extract_features(audio, sr):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccs_processed = np.mean(mfccs.T, axis=0)
    return mfccs_processed

def process_audio(file_path, duration=1800):
    audio, sr = librosa.load(file_path, sr=None, duration=duration)
    features = extract_features(audio, sr)
    return features

In [9]:
def train_model(X_train, y_train, X_test, y_test, num_classes):
    model = Sequential([
        LSTM(64, input_shape=(1, X_train.shape[1])),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
    return model


In [10]:
def predict_emotion(audio_file, model, scaler, le):
    features = process_audio(audio_file)
    features_scaled = scaler.transform(features.reshape(1, -1))
    features_reshaped = features_scaled.reshape((1, 1, features_scaled.shape[1]))
    prediction = model.predict(features_reshaped)
    predicted_class = np.argmax(prediction, axis=1)
    predicted_emotion = le.inverse_transform(predicted_class)[0]
    return predicted_emotion

if __name__ == "__main__":
    youtube_urls = [
    ("https://www.youtube.com/watch?v=9NuRCe-Sl5Q", "depressed"),
    ("https://www.youtube.com/watch?v=yu8T38PbkI8", "anxious"),
    ("https://www.youtube.com/watch?v=_c153L56_Kw", "stressed"),
    ("https://www.youtube.com/watch?v=DwBDY91pHio", "lonely"),
    ("https://www.youtube.com/watch?v=hiFIkCF6SGU", "sad"),
    ("https://www.youtube.com/watch?v=Omoyk7AQW3A", "frustrated"),
    ("https://www.youtube.com/watch?v=EmxDr6PKA68", "angry"),
    ("https://www.youtube.com/watch?v=GucsNugUkgM", "hopeless"),
    ("https://www.youtube.com/watch?v=8MSy7ou9HGQ", "overwhelmed"),
    ("https://www.youtube.com/watch?v=EmxDr6PKA68", "irritated"),
    ("https://www.youtube.com/watch?v=GucsNugUkgM", "disappointed"),
    ("https://www.youtube.com/watch?v=8MSy7ou9HGQ", "helpless"),
    ("https://www.youtube.com/watch?v=EmxDr6PKA68", "resentful"),
    ("https://www.youtube.com/watch?v=ai6xhPFs1cQ", "guilty"),
    ("https://www.youtube.com/watch?v=0IS7ZiGKCV8", "ashamed"),
]


In [11]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Define the output path
output_path = "audio_files"

# Create the directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

all_features = []
all_emotions = []
for url, emotion in youtube_urls:
    try:
        audio_file = download_youtube_audio(url, output_path)
        features = process_audio(audio_file)
        all_features.append(features)
        all_emotions.append(emotion)
        print(f"Processed: {url}")
    except Exception as e:
        print(f"Error processing {url}: {str(e)}")

X = np.array(all_features)
y = np.array(all_emotions)

le = LabelEncoder()
y = le.fit_transform(y)

[youtube] Extracting URL: https://www.youtube.com/watch?v=9NuRCe-Sl5Q
[youtube] 9NuRCe-Sl5Q: Downloading webpage
[youtube] 9NuRCe-Sl5Q: Downloading ios player API JSON
[youtube] 9NuRCe-Sl5Q: Downloading web creator player API JSON
[youtube] 9NuRCe-Sl5Q: Downloading player 5f8f5b0f
[youtube] 9NuRCe-Sl5Q: Downloading m3u8 information
[info] 9NuRCe-Sl5Q: Downloading 1 format(s): 140
[download] Destination: audio_files/kapil sharma crying talking about sunil grover and Shahrukh Khan.m4a
[download] 100% of    2.75MiB in 00:00:00 at 6.54MiB/s   
[FixupM4a] Correcting container of "audio_files/kapil sharma crying talking about sunil grover and Shahrukh Khan.m4a"
[ExtractAudio] Destination: audio_files/kapil sharma crying talking about sunil grover and Shahrukh Khan.mp3
Deleting original file audio_files/kapil sharma crying talking about sunil grover and Shahrukh Khan.m4a (pass -k to keep)
Processed: https://www.youtube.com/watch?v=9NuRCe-Sl5Q
[youtube] Extracting URL: https://www.youtube.com/

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the data for LSTM input
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

def train_model(X_train, y_train, X_test, y_test, num_classes):
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], 1)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
    return model

# Train the model
model = train_model(X_train_lstm, y_train, X_test_lstm, y_test, len(le.classes_))

def predict_emotion(audio_file, model, scaler, le):
    # Load and process the audio file
    features = process_audio(audio_file)
    features_scaled = scaler.transform(features.reshape(1, -1))
    features_reshaped = features_scaled.reshape((1, features_scaled.shape[1], 1))
    prediction = model.predict(features_reshaped)
    predicted_class = np.argmax(prediction, axis=1)
    predicted_emotion = le.inverse_transform(predicted_class)[0]
    return predicted_emotion



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.0833 - loss: 2.7318 - val_accuracy: 0.0000e+00 - val_loss: 2.6866
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.0833 - loss: 2.7090 - val_accuracy: 0.0000e+00 - val_loss: 2.6920
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 0.1667 - loss: 2.6868 - val_accuracy: 0.0000e+00 - val_loss: 2.6963
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - accuracy: 0.0833 - loss: 2.6814 - val_accuracy: 0.0000e+00 - val_loss: 2.6984
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step - accuracy: 0.1667 - loss: 2.6808 - val_accuracy: 0.0000e+00 - val_loss: 2.6957
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.2500 - loss: 2.6430 - val_accuracy: 0.0000e+00 - val_loss: 2.6936
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.0000e+00 - loss: 2.6515 - val_accuracy: 0.0000e+00 - val_loss: 2.6917
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step - accuracy: 0.1667 - loss: 2.6358 - val

In [13]:
# Example usage of the trained model for emotion prediction
audio_file_path = "/content/audio_files/BREAKUP Story of EZ Snippet EXPLAINED. @ezsnippat.mp3"
predicted_emotion = predict_emotion(audio_file_path, model, scaler, le)
print(f"Predicted emotion for {audio_file_path}: {predicted_emotion}")

# Print information about the Vocalysis Bot Prototype
print("Vocalysis Bot Prototype (Mental Health Focus):")
print("- Detects emotions from audio relevant to mental health")
print("- Can assist mental health professionals in assessing emotional states")
print("- Potential applications:")
print("  - Monitoring mental well-being over time")
print("  - Providing early warning signs of emotional distress")
print("  - Supporting remote mental health assessments")
print("  - Enhancing emotional self-awareness for individuals")

print("The Vocalysis bot prototype showcases an innovative approach to leveraging AI and audio analysis for mental health support. By accurately detecting emotions from voice data, it can provide valuable insights to mental health professionals and individuals alike. The bot has the potential to revolutionize mental health assessments, enable early intervention, and improve overall mental well-being.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
Predicted emotion for /content/audio_files/BREAKUP Story of EZ Snippet EXPLAINED. @ezsnippat.mp3: angry
Vocalysis Bot Prototype (Mental Health Focus):
- Detects emotions from audio relevant to mental health
- Can assist mental health professionals in assessing emotional states
- Potential applications:
  - Monitoring mental well-being over time
  - Providing early warning signs of emotional distress
  - Supporting remote mental health assessments
  - Enhancing emotional self-awareness for individuals
The Vocalysis bot prototype showcases an innovative approach to leveraging AI and audio analysis for mental health support. By accurately detecting emotions from voice data, it can provide valuable insights to mental health professionals and individuals alike. The bot has the potential to revolutionize mental health assessments, enable early intervention, and improve overall mental well-being.


In [16]:
def save_model(model, model_path):
    model.save(model_path)
    print(f"Model saved at: {model_path}")

In [19]:

# Save the model
model_path = "vocalysis_model.keras"
save_model(model, model_path)

Model saved at: vocalysis_model.keras


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install streamlit pyngrok plotly pandas

In [1]:
# First, let's create the Streamlit app code
app_code = """
import streamlit as st
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import librosa
import librosa.display
import matplotlib.pyplot as plt
import io
import soundfile as sf

# Placeholder for actual model and processing functions
def process_audio(audio_file):
    # Placeholder for actual audio processing
    return np.random.rand(1, 128)  # Simulated feature vector

def predict_emotion(features):
    emotions = ['Happy', 'Sad', 'Angry', 'Anxious', 'Neutral']
    return np.random.choice(emotions), np.random.rand(5)

st.set_page_config(page_title="Vocalysis", layout="wide")

st.title("Vocalysis: AI-Powered Mental Health Analysis")

# Sidebar for navigation
page = st.sidebar.selectbox("Choose a page", ["Home", "Live Analysis", "Historical Trends", "About"])

if page == "Home":
    st.header("Welcome to Vocalysis")
    st.write("Vocalysis is an innovative AI-powered tool that analyzes voice patterns to detect emotions and potential mental health concerns.")

    st.subheader("Key Features:")
    col1, col2, col3 = st.columns(3)
    with col1:
        st.write("🎙️ Real-time voice analysis")
    with col2:
        st.write("📊 Emotion probability breakdown")
    with col3:
        st.write("📈 Historical trend tracking")

    st.subheader("How it works")
    st.write("1. Record or upload your voice")
    st.write("2. Our AI analyzes vocal patterns")
    st.write("3. Receive instant emotional analysis")
    st.write("4. Track your emotional well-being over time")

elif page == "Live Analysis":
    st.header("Live Voice Analysis")

    upload_option = st.radio("Choose input method:", ("Upload Audio", "Record Live"))

    if upload_option == "Upload Audio":
        audio_file = st.file_uploader("Upload an audio file", type=['wav', 'mp3'])
        if audio_file is not None:
            st.audio(audio_file)
            features = process_audio(audio_file)
            emotion, probabilities = predict_emotion(features)

            st.subheader(f"Detected Emotion: {emotion}")

            fig = px.bar(x=['Happy', 'Sad', 'Angry', 'Anxious', 'Neutral'], y=probabilities)
            fig.update_layout(title="Emotion Probabilities", xaxis_title="Emotions", yaxis_title="Probability")
            st.plotly_chart(fig)

            # Display waveform
            y, sr = librosa.load(audio_file)
            fig, ax = plt.subplots()
            librosa.display.waveshow(y, sr=sr, ax=ax)
            ax.set_title("Audio Waveform")
            st.pyplot(fig)

            # Display spectrogram
            D = librosa.stft(y)
            S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
            fig, ax = plt.subplots()
            img = librosa.display.specshow(S_db, x_axis='time', y_axis='hz', ax=ax)
            fig.colorbar(img, ax=ax, format="%+2.0f dB")
            ax.set_title("Spectrogram")
            st.pyplot(fig)

    else:
        # Placeholder for live recording functionality
        st.write("Live recording feature coming soon!")

elif page == "Historical Trends":
    st.header("Emotional Health Trends")

    # Generate mock data
    dates = pd.date_range(end=pd.Timestamp.now(), periods=30)
    emotions = np.random.choice(['Happy', 'Sad', 'Angry', 'Anxious', 'Neutral'], size=30)
    df = pd.DataFrame({'Date': dates, 'Emotion': emotions})

    fig = px.line(df, x='Date', y='Emotion', title='Emotion Trends Over Time')
    fig.update_traces(mode='lines+markers')
    st.plotly_chart(fig)

    # Emotion distribution
    emotion_counts = df['Emotion'].value_counts()
    fig = px.pie(values=emotion_counts.values, names=emotion_counts.index, title='Emotion Distribution')
    st.plotly_chart(fig)

    st.subheader("Insights")
    st.write("Based on your emotional trends, here are some insights:")
    st.write("- Your dominant emotion this month: " + emotion_counts.index[0])
    st.write("- You've had " + str(emotion_counts['Happy']) + " happy days this month")
    st.write("- Consider scheduling a check-in with a mental health professional if negative emotions persist")

elif page == "About":
    st.header("About Vocalysis")
    st.write("Vocalysis is at the forefront of mental health technology, leveraging artificial intelligence to analyze voice patterns and detect emotions.")

    st.subheader("Our Mission")
    st.write("To provide accessible, non-invasive mental health monitoring tools that empower individuals and support mental health professionals.")

    st.subheader("How It Works")
    st.write("1. Voice Input: Users provide voice samples through audio uploads or live recordings.")
    st.write("2. AI Analysis: Our advanced AI models analyze vocal features like pitch, tone, and rhythm.")
    st.write("3. Emotion Detection: The AI identifies emotional states based on these vocal characteristics.")
    st.write("4. Trend Analysis: Over time, Vocalysis tracks emotional patterns, providing valuable insights into mental well-being.")

    st.subheader("Applications")
    st.write("- Early detection of mental health concerns")
    st.write("- Continuous monitoring for individuals with known mental health conditions")
    st.write("- Support tool for mental health professionals")
    st.write("- Personal emotional awareness and self-improvement")

    st.subheader("Privacy and Ethics")
    st.write("We prioritize user privacy and adhere to strict ethical guidelines in AI development and data handling.")

st.sidebar.write("© 2024 Vocalysis Team")
"""

# Write the app code to a file
with open('app.py', 'w') as f:
    f.write(app_code)

# Install required packages
!pip install streamlit pyngrok pandas numpy plotly matplotlib librosa soundfile

# Set up ngrok
from pyngrok import ngrok
import time

# Set the authtoken
ngrok.set_auth_token("2lEQtqjjBlRy0b4Zu9kiA7KoUap_4xai5bdf8MJrmPYbwnDCg")

# Run the Streamlit app
!streamlit run app.py &>/content/logs.txt &

# Wait for Streamlit to start
time.sleep(5)

# Set up a tunnel to the Streamlit port with explicit configuration
public_url = ngrok.connect(addr="8501", proto="http", bind_tls=True)
print(f'Public URL: {public_url}')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
Public URL: NgrokTunnel: "https://2e8a-34-83-102-148.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def benchmark_model(model, X_test, y_test, class_names):
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    # Classification report
    print(classification_report(y_true_classes, y_pred_classes, target_names=class_names))

    # Confusion matrix
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # Per-class accuracy
    per_class_accuracy = cm.diagonal() / cm.sum(axis=1)
    for i, accuracy in enumerate(per_class_accuracy):
        print(f"{class_names[i]} accuracy: {accuracy:.4f}")

# Usage example:
# benchmark_model(model, X_test, y_test, class_names=['class1', 'class2', ...])